In [1]:
# TFM_04_data_cleaning_and_split.ipynb

In [2]:
import pandas as pd
import numpy as np
from typing import List, Optional, Tuple
from collections import defaultdict
import os

pd.set_option('display.float_format', lambda x: '{:.12f}'.format(x))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 0)

In [3]:
try:
    df = pd.read_csv('df_final_target_20250206.csv')
    df2 = pd.read_csv('indicators/df2.csv')
except FileNotFoundError as e:
    print(f"Archivo no encontrado: {e.filename}")
    exit()
except Exception as e:
    print(f"Error cargando datos.")
    exit()

In [4]:
def analizar_tipos_columnas_corto(df, df_nombre="DataFrame"):
    """Analiza y muestra tipos de columnas y cuenta."""

    if not isinstance(df, pd.DataFrame):
        print(f"{df_nombre} no es un DataFrame.")
        return

    resultados = {
        'booleano': [col for col in df.columns if pd.api.types.is_bool_dtype(df[col])],
        'numérico': [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])],
        'timestamp': [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])],
        'string': [col for col in df.columns if df[col].dtype == 'object'],
        'otro': [col for col in df.columns if not any([
            pd.api.types.is_bool_dtype(df[col]),
            pd.api.types.is_numeric_dtype(df[col]),
            pd.api.types.is_datetime64_any_dtype(df[col]),
            df[col].dtype == 'object'
        ])]
    }

    if not resultados:
         print(f"No se encontraron columnas en {df_nombre}.")
         return

    for tipo, columnas in resultados.items():
        print(f"Tipo: {tipo.capitalize()}")
        print(f"  Número: {len(columnas)}")
        if columnas:
            max_cols_a_mostrar = 5
            if len(columnas) > max_cols_a_mostrar:
                 print(f"  Columnas: {columnas[:max_cols_a_mostrar]}...")
            else:
                 print(f"  Columnas: {columnas}")
        else:
            print("  No hay columnas.")
        print("-" * 20)


In [5]:
analizar_tipos_columnas_corto(df)
analizar_tipos_columnas_corto(df2, df_nombre="df2")

Tipo: Booleano
  Número: 13
  Columnas: ['Doji', 'Hammer', 'HangingMan', 'BullishEngulfing', 'BearishEngulfing']...
--------------------
Tipo: Numérico
  Número: 487
  Columnas: ['open', 'high', 'low', 'close', 'volume']...
--------------------
Tipo: Timestamp
  Número: 0
  No hay columnas.
--------------------
Tipo: String
  Número: 2
  Columnas: ['timestamp', 'symbol']
--------------------
Tipo: Otro
  Número: 0
  No hay columnas.
--------------------
Tipo: Booleano
  Número: 13
  Columnas: ['Doji', 'Hammer', 'HangingMan', 'BullishEngulfing', 'BearishEngulfing']...
--------------------
Tipo: Numérico
  Número: 467
  Columnas: ['open', 'high', 'low', 'close', 'volume']...
--------------------
Tipo: Timestamp
  Número: 0
  No hay columnas.
--------------------
Tipo: String
  Número: 2
  Columnas: ['timestamp', 'symbol']
--------------------
Tipo: Otro
  Número: 0
  No hay columnas.
--------------------


In [6]:
def convertir(df, timestamp_col='timestamp'):
    """Convierte columna timestamp a datetime y booleanos a int."""
    # Args: df (pd.DataFrame), timestamp_col (str)
    # Return: pd.DataFrame
    if not isinstance(df, pd.DataFrame):
        print("Entrada no es DataFrame.")
        return df

    df_modificado = df.copy()

    try:
        if timestamp_col in df_modificado.columns:
            df_modificado[timestamp_col] = pd.to_datetime(df_modificado[timestamp_col], errors='ignore')
        else:
             print(f"Columna '{timestamp_col}' no existe.")
    except Exception:
        print(f"Fallo al convertir '{timestamp_col}'.")

    bool_cols = df_modificado.select_dtypes(include='bool').columns
    if not bool_cols.empty:
        df_modificado[bool_cols] = df_modificado[bool_cols].astype(int)

    return df_modificado

In [7]:
df = convertir(df)
df2 = convertir(df2)

In [8]:
df = df.sort_values('timestamp')
if 'symbol' in df.columns:
    symbols = df['symbol'].unique()
    print(f"Símbolos: {len(symbols)}")
else:
    symbols = ['unknown_symbol']

Símbolos: 310


In [9]:
missing_values = df.isnull().sum()
total_missing = missing_values.sum()
print(f"Valores faltantes: {total_missing}")
missing_per_column = missing_values[missing_values > 0]
if not missing_per_column.empty:
    print("Faltantes por columna:")
    print(missing_per_column)

Valores faltantes: 205
Faltantes por columna:
timestamp    205
dtype: int64


In [10]:
print(f"Shape df: {df.shape}")
print(f"Shape df2: {df2.shape}")

columnas_con_nan = df.columns[df.isna().any()]

if len(columnas_con_nan) > 0:
    print("Columnas con NaN en df:")
    print(columnas_con_nan.tolist())

Shape df: (432450, 489)
Shape df2: (1, 469)
Columnas con NaN en df:
['timestamp']


In [11]:
columnas_unicas_df = df.columns.difference(df2.columns)
columnas_unicas_df2 = df2.columns.difference(df.columns)

print(f"Columnas únicas df:", columnas_unicas_df.tolist())
print(f"Columnas únicas df2:", columnas_unicas_df2.tolist())

Columnas únicas df: ['ATR_20_RollingMean', 'MFI_20', 'close_lead_1', 'close_lead_10', 'close_lead_11', 'close_lead_12', 'close_lead_13', 'close_lead_14', 'close_lead_15', 'close_lead_2', 'close_lead_3', 'close_lead_4', 'close_lead_5', 'close_lead_6', 'close_lead_7', 'close_lead_8', 'close_lead_9', 'future_max_increase_capped', 'is_sinusoid_shape', 'target']
Columnas únicas df2: []


In [12]:
drop_columns = [
    'ATR_20_RollingMean', 'MFI_20', 'is_sinusoid_shape', 'close_lead_1',
    'close_lead_2', 'close_lead_3', 'close_lead_4', 'close_lead_5',
    'close_lead_6', 'close_lead_7', 'close_lead_8', 'close_lead_9',
    'close_lead_10', 'close_lead_11', 'close_lead_12', 'close_lead_13',
    'close_lead_14', 'close_lead_15'
]

df.drop(columns=drop_columns, inplace=True, errors='ignore')
df2.drop(columns=drop_columns, inplace=True, errors='ignore')

In [13]:
if 'df' in locals() and isinstance(df, pd.DataFrame) and \
   'df2' in locals() and isinstance(df2, pd.DataFrame):

    columnas_unicas_df_post_drop = df.columns.difference(df2.columns)
    print(f"Columnas únicas en df post-drop: {columnas_unicas_df_post_drop.tolist()}")

else:
    if 'df' not in locals() or not isinstance(df, pd.DataFrame):
         print("Variable df no válida.")
    if 'df2' not in locals() or not isinstance(df2, pd.DataFrame):
         print("Variable df2 no válida.")

Columnas únicas en df post-drop: ['future_max_increase_capped', 'target']


In [14]:
if 'df2' in locals() and isinstance(df2, pd.DataFrame) and not df2.empty:
    fila_df2 = df2.iloc[0]

    if 'timestamp' in df.columns and 'symbol' in df.columns and \
       'timestamp' in fila_df2.index and 'symbol' in fila_df2.index:

        fila_df = df[(df['timestamp'] == fila_df2['timestamp']) & (df['symbol'] == fila_df2['symbol'])]

        if not fila_df.empty:
            fila_df_seleccionada = fila_df.iloc[0]
            fila_df_dict = fila_df_seleccionada.to_dict()
            fila_df2_dict = fila_df2.to_dict()

            diferencias = {}
            columnas_comunes = set(fila_df_dict.keys()) & set(fila_df2_dict.keys())

            for col in columnas_comunes:
                valor_df2 = fila_df2_dict[col]
                valor_df = fila_df_dict[col]

                if isinstance(valor_df2, float) and isinstance(valor_df, float):
                    if not np.isclose(valor_df2, valor_df, atol=1e-8, equal_nan=True):
                        diferencias[col] = (valor_df2, valor_df)
                elif pd.isna(valor_df2) and pd.isna(valor_df):
                     continue
                else:
                    if valor_df2 != valor_df:
                        diferencias[col] = (valor_df2, valor_df)

            if diferencias:
                print("Diferencias fila 0 (df vs df2):")
                for col, (val_df2, val_df) in diferencias.items():
                    print(f"  {col}: df2={val_df2}, df={val_df}")
            else:
                print("Filas coincidentes.")
        else:
            print("No se encontró fila coincidente en df.")
    else:
         print("Faltan columnas 'timestamp' o 'symbol'.")
else:
     print("DataFrame df2 no válido o vacío.")

Filas coincidentes.


In [15]:
def prepare_dataframe_types(
    df: pd.DataFrame,
    target_cols: Optional[List[str]] = None,
    allowed_object_cols: Optional[List[str]] = None
) -> pd.DataFrame:
    """Prepara y verifica tipos de datos del DataFrame."""

    if not isinstance(df, pd.DataFrame):
        print("Entrada no es DataFrame.")
        return df

    df_processed = df.copy()
    print(f"Preparando tipos. Shape: {df_processed.shape}")

    bool_cols = df_processed.select_dtypes(include='bool').columns
    if not bool_cols.empty:
        print(f"Convirtiendo booleanos: {list(bool_cols)}")
        try:
            df_processed[bool_cols] = df_processed[bool_cols].astype(int)
            if not df_processed.select_dtypes(include='bool').columns.empty:
                 print("Fallo al convertir booleanos.")
        except Exception:
            print("Error convirtiendo booleanos.")
    else:
        print("No hay columnas booleanas.")

    object_cols = df_processed.select_dtypes(include='object').columns
    permitidas = allowed_object_cols if allowed_object_cols is not None else []
    problematic_object_cols = [col for col in object_cols if col not in permitidas]

    if problematic_object_cols:
        print(f"Columnas 'object' no permitidas: {problematic_object_cols}.")
    else:
         print("No hay columnas 'object' no permitidas.")

    if target_cols:
        print("Verificando columnas objetivo.")
        targets_existentes = []
        all_numeric = True
        for col in target_cols:
            if col in df_processed.columns:
                targets_existentes.append(col)
                dtype = df_processed[col].dtype
                if not pd.api.types.is_numeric_dtype(df_processed[col]):
                    print(f"  Target '{col}' no numérico (Tipo: {dtype}).")
                    all_numeric = False
            else:
                print(f"  Target '{col}' no encontrado.")
                all_numeric = False

        if all_numeric and len(targets_existentes) == len(target_cols):
            print("Targets numéricos y presentes.")
        elif not targets_existentes:
             print(f"Targets no encontrados.")

    return df_processed

In [16]:
cols_objetivo = ['target', 'future_max_increase_capped']
cols_object_ok = ['symbol','timestamp']

df = prepare_dataframe_types(
    df,
    target_cols=cols_objetivo,
    allowed_object_cols=cols_object_ok
)
print(f"Dimensiones df: {df.shape}")

df2 = prepare_dataframe_types(
    df2,
    target_cols=cols_objetivo,
    allowed_object_cols=cols_object_ok
)
print(f"Dimensiones df2: {df2.shape}")

Preparando tipos. Shape: (432450, 471)
No hay columnas booleanas.
No hay columnas 'object' no permitidas.
Verificando columnas objetivo.
Targets numéricos y presentes.
Dimensiones df: (432450, 471)
Preparando tipos. Shape: (1, 469)
No hay columnas booleanas.
No hay columnas 'object' no permitidas.
Verificando columnas objetivo.
  Target 'target' no encontrado.
  Target 'future_max_increase_capped' no encontrado.
Targets no encontrados.
Dimensiones df2: (1, 469)


In [17]:
df3 = df.copy()
df4 = df2.copy()

In [18]:
if 'timestamp' in df3.columns:
    df3['timestamp'] = pd.to_datetime(df3['timestamp'], errors='coerce')
else:
    print("Columna 'timestamp' no encontrada en df3.")

expected_bool_cols = df3.select_dtypes(include=['bool']).columns.tolist()
if expected_bool_cols:
     print(f"Columnas booleanas inesperadas en df3: {expected_bool_cols}")

In [19]:
def detectar_columnes_iguals(df, num_inicial=3, num_final=500):
    """Detecta columnas idénticas en un DataFrame."""

    iguals = defaultdict(list)
    columnes = df.columns

    if len(df) == 0:
        return dict(iguals)

    df_mini = df.iloc[:min(num_inicial, len(df))].copy()

    grupos_iniciales = defaultdict(list)
    for col in columnes:
        try:
            col_hash = hash(str(df_mini[col].values.tobytes()))
            grupos_iniciales[col_hash].append(col)
        except Exception:
            continue

    grupos_candidatos = [cols for cols in grupos_iniciales.values() if len(cols) > 1]

    muestras = [10, 50, 100, num_final] if num_final > 100 else [min(len(df), num_final)]
    muestras = [m for m in muestras if m <= len(df)]

    for num_muestra in muestras:
        if not grupos_candidatos:
            break

        df_muestra = df.iloc[:num_muestra].copy()
        nuevos_grupos = []
        for grupo in grupos_candidatos:
            subgrupos = defaultdict(list)
            for col in grupo:
                try:
                    col_hash = hash(str(df_muestra[col].values.tobytes()))
                    subgrupos[col_hash].append(col)
                except Exception:
                    continue
            nuevos_grupos.extend([cols for cols in subgrupos.values() if len(cols) > 1])
        grupos_candidatos = nuevos_grupos

    columnes_processades = set()
    for grupo in grupos_candidatos:
        if not grupo: continue
        col_base = grupo[0]
        if col_base in columnes_processades: continue

        for col_comp in grupo[1:]:
            if col_comp in columnes_processades:
                continue

            if df[col_base].equals(df[col_comp]):
                iguals[col_base].append(col_comp)
                columnes_processades.add(col_comp)

    return dict(iguals)

In [20]:
def detectar_relacions_factorials(df, num_inicial=3, num_final=500, tolerancia=1e-6):
    """Detecta relaciones factoriales entre columnas numéricas."""

    relacions = defaultdict(list)
    columnes_numeriques = df.select_dtypes(include=np.number).columns

    if len(df) == 0 or len(columnes_numeriques) < 2:
        return dict(relacions)

    df_mini = df.iloc[:min(num_inicial, len(df))].copy()

    columnes_no_constantes = [
        col for col in columnes_numeriques
        if df_mini[col].nunique(dropna=False) > 1
    ]

    firmas = {}
    for col in columnes_no_constantes:
        vals = df_mini[col].replace([np.inf, -np.inf], np.nan).dropna().values
        if len(vals) > 0:
            idx_no_zero = np.where(vals != 0)[0]
            if len(idx_no_zero) > 0:
                first_nonzero = vals[idx_no_zero[0]]

                firma = tuple(np.round(vals / first_nonzero, 6))
                firmas[col] = firma

    grupos_firmas = defaultdict(list)
    for col, firma in firmas.items():
        grupos_firmas[firma].append(col)

    pares_candidatos = []
    for grupo in grupos_firmas.values():
        if len(grupo) > 1:
            for i in range(len(grupo)):
                for j in range(i + 1, len(grupo)):
                    pares_candidatos.append((grupo[i], grupo[j]))

    muestras = [10, 50, 100, num_final] if num_final > 100 else [min(len(df), num_final)]
    muestras = [m for m in muestras if m <= len(df)]
    pares_confirmados_temporal = []

    for num_muestra in muestras:
        if not pares_candidatos:
            break

        df_muestra = df.iloc[:num_muestra].copy()
        nuevos_pares_confirmados = []
        pares_para_siguiente_ronda = []

        for col_a_nom, col_b_nom in pares_candidatos:
            col_a = df_muestra[col_a_nom]
            col_b = df_muestra[col_b_nom]
            factor_encontrado = None
            relacion = None 

            with np.errstate(divide='ignore', invalid='ignore'):
                factors_ab = (col_a / col_b).replace([np.inf, -np.inf], np.nan).dropna()
            if len(factors_ab) > 0 and factors_ab.nunique() == 1:
                 factor_encontrado = factors_ab.iloc[0]
                 relacion = 'ab'

            if factor_encontrado is None:
                 with np.errstate(divide='ignore', invalid='ignore'):
                     factors_ba = (col_b / col_a).replace([np.inf, -np.inf], np.nan).dropna()
                 if len(factors_ba) > 0 and factors_ba.nunique() == 1:
                     factor_encontrado = factors_ba.iloc[0]
                     relacion = 'ba'

            if factor_encontrado is not None:
                 nuevos_pares_confirmados.append((col_a_nom, col_b_nom, factor_encontrado, relacion))
                 pares_para_siguiente_ronda.append((col_a_nom, col_b_nom))

        pares_candidatos = pares_para_siguiente_ronda
        pares_confirmados_temporal = nuevos_pares_confirmados 
        
        if not pares_candidatos:
             break

    relacions_trobades = set()
    for col_a_nom, col_b_nom, factor, tipo in pares_confirmados_temporal:
        if (col_a_nom, col_b_nom) in relacions_trobades or (col_b_nom, col_a_nom) in relacions_trobades:
            continue

        col_a_full = df[col_a_nom].fillna(0).values
        col_b_full = df[col_b_nom].fillna(0).values

        try:
            if tipo == 'ab':
                comparison = np.isclose(col_a_full, factor * col_b_full, atol=tolerancia, rtol=tolerancia, equal_nan=True)
                if np.all(comparison):
                    relacions[col_a_nom].append((col_b_nom, factor))
                    relacions_trobades.add((col_a_nom, col_b_nom))
            else: 
                comparison = np.isclose(col_b_full, factor * col_a_full, atol=tolerancia, rtol=tolerancia, equal_nan=True)
                if np.all(comparison):
                    relacions[col_b_nom].append((col_a_nom, factor))
                    relacions_trobades.add((col_b_nom, col_a_nom))
        except Exception:

             continue

    return dict(relacions)

In [21]:
df_analisis = df3.copy() 

print("Detectando columnas iguales...")
columnes_iguals = detectar_columnes_iguals(df_analisis)

print("Detectando relaciones factoriales...")
relacions_factorials = detectar_relacions_factorials(df_analisis)

print("--- Resultados ---")
if columnes_iguals:
    print("Columnas idénticas:")
    for col, duplicats in columnes_iguals.items():
        print(f"  {col}: {duplicats}")
else:
    print("No hay columnas idénticas.")

if relacions_factorials:
    print("Relaciones factoriales:")
    for col, relacionats in relacions_factorials.items():
        rel_formateada = [(nom, f"{factor:.6g}") for nom, factor in relacionats]
        print(f"  {col}: {rel_formateada}")
else:
    print("No hay relaciones factoriales.")

columnes_a_suprimir = [
    'open_lag_0', 'high_lag_0', 'low_lag_0', 'close_lag_0', 'close_lead_0', 'volume_lag_0',
    'quote_asset_volume_lag_0', 'number_of_trades_lag_0', 'taker_buy_base_asset_volume_lag_0',
    'taker_buy_quote_asset_volume_lag_0', 'TakerBuyQuoteVolume', 'BB_Middle_20', 'HangingMan',
    'dx_smoothed_lag_1', 'ATR_14_lag_0', 'ATR_14_lag_1', 'ATR_14_lag_2', 'ATR_14_lag_3',
    'ATR_14_lag_4', 'ATR_14_lag_5', 'ATR_14_lag_6', 'ATR_14_lag_7', 'ATR_14_lag_8',
    'ATR_14_lag_9', 'ATR_14_lag_10', 'ATR_14_lag_11', 'ATR_14_lag_12', 'ATR_14_lag_13',
    'ATR_14_RollingMean'
]

df3 = df3.drop(columns=columnes_a_suprimir, errors='ignore')
df4 = df4.drop(columns=columnes_a_suprimir, errors='ignore')

print(f"Columnas eliminadas (si existían). Shape df3: {df3.shape}, Shape df4: {df4.shape}")

Detectando columnas iguales...
Detectando relaciones factoriales...
--- Resultados ---
Columnas idénticas:
  open: ['open_lag_0']
  high: ['high_lag_0']
  low: ['low_lag_0']
  close: ['close_lag_0', 'close_lead_0']
  volume: ['volume_lag_0']
  quote_asset_volume: ['quote_asset_volume_lag_0']
  number_of_trades: ['number_of_trades_lag_0']
  taker_buy_base_asset_volume: ['taker_buy_base_asset_volume_lag_0']
  taker_buy_quote_asset_volume: ['taker_buy_quote_asset_volume_lag_0', 'TakerBuyQuoteVolume']
  SMA_20: ['BB_Middle_20']
  Hammer: ['HangingMan']
  ADX: ['dx_smoothed_lag_1']
  ATR_14: ['ATR_14_lag_0', 'ATR_14_lag_1', 'ATR_14_lag_2', 'ATR_14_lag_3', 'ATR_14_lag_4', 'ATR_14_lag_5', 'ATR_14_lag_6', 'ATR_14_lag_7', 'ATR_14_lag_8', 'ATR_14_lag_9', 'ATR_14_lag_10', 'ATR_14_lag_11', 'ATR_14_lag_12', 'ATR_14_lag_13', 'ATR_14_RollingMean']
Relaciones factoriales:
  open: [('open_lag_0', '1')]
  high: [('high_lag_0', '1')]
  low: [('low_lag_0', '1')]
  close: [('close_lag_0', '1'), ('close_lea

In [22]:
def filtrar_simbolos_por_targets(
    df: pd.DataFrame,
    col_simbolo: str = 'symbol',
    col_target: str = 'target',
    valor_target_positivo: int = 1,
    min_targets_positivos: int = 1
) -> Tuple[pd.DataFrame, List[str], List[str]]:
    """Filtra DataFrame manteniendo símbolos con un mínimo de targets positivos."""

    if col_simbolo not in df.columns or col_target not in df.columns:
        print(f"Columnas requeridas no encontradas.")
        simbolos_presentes = list(df[col_simbolo].unique()) if col_simbolo in df.columns else []
        return df, simbolos_presentes, []

    simbolos_originales = df[col_simbolo].unique()
    print(f"Evaluando {len(simbolos_originales)} símbolos.")

    conteo_por_simbolo = df[df[col_target] == valor_target_positivo].groupby(col_simbolo)[col_target].count()
    simbolos_a_mantener = conteo_por_simbolo[conteo_por_simbolo >= min_targets_positivos].index.tolist()
    simbolos_eliminados = list(set(simbolos_originales) - set(simbolos_a_mantener))

    print("Filtrado completado.")
    print(f"  Símbolos mantenidos: {len(simbolos_a_mantener)}")
    print(f"  Símbolos eliminados: {len(simbolos_eliminados)}")

    if not simbolos_a_mantener:
        print("Ningún símbolo cumple el criterio.")
        return pd.DataFrame(columns=df.columns), [], list(simbolos_originales)

    df_filtrado = df[df[col_simbolo].isin(simbolos_a_mantener)].copy()
    print(f"Shape antes del filtro: {df.shape}")
    print(f"Shape después del filtro: {df_filtrado.shape}")

    return df_filtrado, simbolos_a_mantener, simbolos_eliminados



In [23]:
columna_id = 'symbol'
columna_objetivo = 'target'
valor_positivo = 1
minimo_requerido = 1

df_filtrado_targets, simbolos_validos, simbolos_fuera = filtrar_simbolos_por_targets(
    df=df3,
    col_simbolo=columna_id,
    col_target=columna_objetivo,
    valor_target_positivo=valor_positivo,
    min_targets_positivos=minimo_requerido
)

df3 = df_filtrado_targets
print("DataFrame 'df3' actualizado tras filtro.")

if df3.empty:
    print("DataFrame 'df3' vacío después del filtrado.")

Evaluando 310 símbolos.
Filtrado completado.
  Símbolos mantenidos: 77
  Símbolos eliminados: 233
Shape antes del filtro: (432450, 442)
Shape después del filtro: (107415, 442)
DataFrame 'df3' actualizado tras filtro.


In [24]:
def dividir_dataframe_train_test(df, columna_tiempo, columnas_para_quitar,
                                 ratio_train, carpeta_salida, prefijo):
    """Divide DataFrame en train/test por tiempo y guarda en CSV."""


    if not isinstance(df, pd.DataFrame) or df.empty:
        print("DataFrame vacío o inválido.")
        return

    df_copia = df.copy()

    if columna_tiempo in df_copia.columns:
        print(f"Ordenando por '{columna_tiempo}'")
        df_copia = df_copia.sort_values(by=columna_tiempo).reset_index(drop=True)
    else:
        print(f"Columna '{columna_tiempo}' no encontrada para ordenar.")


    if columnas_para_quitar:
        cols_existentes = [col for col in columnas_para_quitar if col in df_copia.columns]
        if cols_existentes:
            df_copia = df_copia.drop(columns=cols_existentes)
            print(f"Columnas eliminadas: {cols_existentes}")

    n_filas = len(df_copia)
    n_train = int(n_filas * ratio_train)
    n_test = n_filas - n_train
    print(f"Filas: {n_filas}, Train: {n_train}, Test: {n_test}")

    if n_train <= 0 or n_test <= 0:
        print(f"División inválida (Train={n_train}, Test={n_test}).")
        return

    df_train = df_copia.iloc[:n_train]
    df_test = df_copia.iloc[n_train:]
    print(f"Shape df_train: {df_train.shape}, Shape df_test: {df_test.shape}")

    try:
        os.makedirs(carpeta_salida, exist_ok=True)
    except OSError as e:
        print(f"Error creando carpeta '{carpeta_salida}'.")
        return

    nombre_train = os.path.join(carpeta_salida, f"{prefijo}_train_completo.csv")
    nombre_test = os.path.join(carpeta_salida, f"{prefijo}_test_completo.csv")
    try:
        df_train.to_csv(nombre_train, index=False) 
        df_test.to_csv(nombre_test, index=False)
        print(f"Guardado: {nombre_train}")
        print(f"Guardado: {nombre_test}")
    except Exception as e:
        print(f"Error guardando archivos CSV.")
        return

In [25]:
columnas_a_ignorar = []
carpeta_destino = "datos_divididos"
nombre_base = "datos_financieros"
proporcion_entrenamiento = 0.8

dividir_dataframe_train_test(
    df = df3,
    columna_tiempo = 'timestamp',
    columnas_para_quitar = columnas_a_ignorar,
    ratio_train = proporcion_entrenamiento,
    carpeta_salida = carpeta_destino,
    prefijo = nombre_base
)

print(f"\nProceso de división finalizado. Archivos en '{carpeta_destino}'")

Ordenando por 'timestamp'
Filas: 107415, Train: 85932, Test: 21483
Shape df_train: (85932, 442), Shape df_test: (21483, 442)
Guardado: datos_divididos\datos_financieros_train_completo.csv
Guardado: datos_divididos\datos_financieros_test_completo.csv

Proceso de división finalizado. Archivos en 'datos_divididos'
